# Lab | BabyAGI with agent

**Change the planner objective below by changing the objective and the associated prompts and potential tools and agents - Wear your creativity and AI engineering hats
You can't get this wrong!**

You would need the OpenAI API KEY and the [SerpAPI KEY](https://serpapi.com/manage-api-keyhttps://serpapi.com/manage-api-key) to run this lab.


## BabyAGI with Tools

This notebook builds on top of [baby agi](baby_agi.html), but shows how you can swap out the execution chain. The previous execution chain was just an LLM which made stuff up. By swapping it out with an agent that has access to tools, we can hopefully get real reliable information

## Install and Import Required Modules

In [1]:
pip install langchain_community langchain_experimental langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 781.4 kB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
from typing import Optional

from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_experimental.autonomous_agents import BabyAGI
from langchain_openai import OpenAI, OpenAIEmbeddings

## Connect to the Vector Store

Depending on what vectorstore you use, this step may look different.

In [2]:
# %pip install faiss-cpu > /dev/null
# %pip install google-search-results > /dev/null
from langchain.docstore import InMemoryDocstore
from langchain_community.vectorstores import FAISS

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
SERPAPI_API_KEY = os.getenv('SERPAPI_API_KEY')

In [9]:
pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 219.6 kB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
from langchain.vectorstores.faiss import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.docstore import InMemoryDocstore
import faiss # type: ignore

embedding_model = OpenAIEmbeddings()

embedding_size = 1536  
index = faiss.IndexFlatL2(embedding_size)

vectorstore = FAISS(
    embedding_model.embed_query,
    index,
    InMemoryDocstore({}),
    {}
)


/var/folders/8p/lhfbj_0n501d3lhdqz4xc0dh0000gn/T/ipykernel_1843/3411753324.py:6: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding_model = OpenAIEmbeddings()
`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


## Define the Chains

BabyAGI relies on three LLM chains:
- Task creation chain to select new tasks to add to the list
- Task prioritization chain to re-prioritize tasks
- Execution Chain to execute the tasks


NOTE: in this notebook, the Execution chain will now be an agent.

In [9]:
pip install python-dotenv



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install google-search-results

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32091 sha256=191b863459e8f0ce9c8ddebf37559bca3269db2b5280c98d08ed71cf83e11cab
  Stored in directory: /Users/amira/Library/Caches/pip/wheels/6e/42/3e/aeb691b02cb7175ec70e2da04b5658d4739d2b41e5f73cd06f
Successfully built google-search-results

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [12]:
from langchain.agents import AgentExecutor, Tool, ZeroShotAgent
from langchain.chains import LLMChain
from langchain_community.utilities import SerpAPIWrapper
from langchain_openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()

todo_prompt = PromptTemplate.from_template(
    "You are a planner who is an expert at coming up with a todo list for a given objective. Come up with a todo list for this objective: {objective}"
)
todo_chain = LLMChain(llm=OpenAI(temperature=0), prompt=todo_prompt)
search = SerpAPIWrapper()
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events",
    ),
    Tool(
        name="TODO",
        func=todo_chain.run,
        description="useful for when you need to come up with todo lists. Input: an objective to create a todo list for. Output: a todo list for that objective. Please be very clear what the objective is!",
    ),
]


prefix = """You are an AI who performs one task based on the following objective: {objective}. Take into account these previously completed tasks: {context}."""
suffix = """Question: {task}
{agent_scratchpad}"""
prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["objective", "task", "context", "agent_scratchpad"],
)

In [13]:
llm = OpenAI(temperature=0)
llm_chain = LLMChain(llm=llm, prompt=prompt)
tool_names = [tool.name for tool in tools]
agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)

/var/folders/8p/lhfbj_0n501d3lhdqz4xc0dh0000gn/T/ipykernel_1843/3064747660.py:4: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)


### Run the BabyAGI

Now it's time to create the BabyAGI controller and watch it try to accomplish your objective.

In [14]:
OBJECTIVE = "Write a weather report for SF today"

In [15]:
# Logging of LLMChains
verbose = False
# If None, will keep on going forever
max_iterations: Optional[int] = 3
baby_agi = BabyAGI.from_llm(
    llm=llm,
    vectorstore=vectorstore,
    task_execution_chain=agent_executor,
    verbose=verbose,
    max_iterations=max_iterations,
)

In [16]:
baby_agi({"objective": OBJECTIVE})

/var/folders/8p/lhfbj_0n501d3lhdqz4xc0dh0000gn/T/ipykernel_1843/3867971396.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  baby_agi({"objective": OBJECTIVE})



*****TASK LIST*****

1: Make a todo list

*****NEXT TASK*****

1: Make a todo list


> Entering new AgentExecutor chain...
Thought: I should use the TODO action to create a todo list
Action: TODO
Action Input: "Create a todo list for writing a weather report for SF today"
Observation: 

1. Gather all necessary information: Start by collecting all the necessary information for writing a weather report for San Francisco today. This includes current temperature, humidity, wind speed, and any other relevant data.

2. Check weather forecast: Before writing the report, check the weather forecast for San Francisco today to get an idea of what to expect. This will help you accurately report on the current weather conditions.

3. Determine target audience: Consider who will be reading the weather report and tailor your writing style and language accordingly. A report for a local news station may be different from one for a scientific journal.

4. Create an outline: Organize your thoughts and i

{'objective': 'Write a weather report for SF today'}

In [17]:
# 1. Define objective
OBJECTIVE = "Write a weather report for SF today"

# 2. Enable or disable verbose logging 
verbose = False

# 3. Set the maximum number of iterations BabyAGI should perform (None = run forever)
max_iterations: Optional[int] = 3

# 4. Initialize BabyAGI with the required components
# - llm: the language model you’re using (e.g., OpenAI’s GPT)
# - vectorstore: memory to keep track of previous tasks and context
# - task_execution_chain: the executor that actually performs tasks
# - verbose: whether to show detailed logs
# - max_iterations: how many task loops BabyAGI will go through

baby_agi = BabyAGI.from_llm(
    llm=llm,
    vectorstore=vectorstore,
    task_execution_chain=agent_executor,
    verbose=verbose,
    max_iterations=max_iterations,
)

# 5. Start the agent by giving it the main objective
baby_agi({"objective": OBJECTIVE})


*****TASK LIST*****

1: Make a todo list

*****NEXT TASK*****

1: Make a todo list


> Entering new AgentExecutor chain...
Thought: The objective is to create a todo list, so I should gather information on what tasks need to be completed.
Action: TODO
Action Input: Objective: Create a todo list
Observation: 

1. Define the purpose of the todo list: Before creating a todo list, it's important to understand why you need it. Is it for personal use or for a specific project? This will help guide the rest of the list.

2. Brainstorm tasks: Take some time to brainstorm all the tasks that need to be included in the todo list. This can include both short-term and long-term tasks.

3. Prioritize tasks: Once you have a list of tasks, prioritize them based on urgency and importance. This will help you focus on the most important tasks first.

4. Break down larger tasks: If there are any tasks that seem overwhelming or too big, break them down into smaller, more manageable tasks.

5. Set deadline

ValueError: Tried to add ids that already exist: {'result_1_0'}